In [1]:
!pip install pandas
!pip install langchain
!pip install langchain-community
!pip install numpy
!pip install langchain_google_genai

  Using cached pandas-2.2.3-cp312-cp312-macosx_11_0_arm64.whl.metadata (89 kB)
  Using cached pytz-2024.2-py2.py3-none-any.whl.metadata (22 kB)
  Using cached tzdata-2024.2-py2.py3-none-any.whl.metadata (1.4 kB)
Using cached pandas-2.2.3-cp312-cp312-macosx_11_0_arm64.whl (11.4 MB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.1/5.1 MB 1.9 MB/s eta 0:00:00a 0:00:01
Using cached pytz-2024.2-py2.py3-none-any.whl (508 kB)
Using cached tzdata-2024.2-py2.py3-none-any.whl (346 kB)
  Using cached PyYAML-6.0.2-cp312-cp312-macosx_11_0_arm64.whl.metadata (2.1 kB)
  Using cached SQLAlchemy-2.0.35-cp312-cp312-macosx_11_0_arm64.whl.metadata (9.6 kB)
  Using cached langchain_text_splitters-0.3.0-py3-none-any.whl.metadata (2.3 kB)
  Using cached numpy-1.26.4-cp312-cp312-macosx_11_0_arm64.whl.metadata (61 kB)
  Using cached pydantic-2.9.2-py3-none-any.whl.metadata (149 kB)
  Using cached requests-2.32.3-py3-none-any.whl.metadata (4.6 kB)
  Using cached tenacity-8.5.0-py3-none-any.whl.metadata (1.2 kB)


In [2]:
import pandas as pd
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Pinecone

In [3]:
data="""
Customer ID,Customer Name,Customer Email,Phone Number,Product ID,Product Name,Purchase Date,Quantity,Price per Unit,Total Price,Payment Method,Shipping Address,Feedback Score,Discount Applied,Transaction Status,Purchase Type
1,Alex Anderson,kramos@baird.com,469.331.1356,103,Rag Type C,2024-09-03,1,6.25,6.25,PayPal,"51705 Ryan Orchard, Williamsbury, WV 27207",4,15%,Successful,In-store
2,Adrian Daniels,gregory96@hotmail.com,001-103-780-1056x30957,104,Rag Type D,2024-03-24,11,4.75,52.25,Cash,"USCGC Walker, FPO AE 33074",4,10%,Successful,Online
3,Misty Craig,brian28@martin-pham.com,168-782-7001x2148,103,Rag Type C,2024-03-09,5,6.25,31.25,Credit Card,"146 Hudson Divide, Port Carol, LA 69845",1,15%,Successful,Online
4,Joseph Hess IV,taylorlori@yahoo.com,+1-072-724-4300,103,Rag Type C,2024-07-24,2,6.25,12.5,Cash,"1484 Barry Brooks Apt. 251, East David, TN 35123",1,15%,Failed,Online
5,Nicole Ochoa,livingstonjustin@hotmail.com,(986)885-9494x384,102,Rag Type B,2024-05-15,10,3.5,35.0,PayPal,"4935 Rachel Pines, Nguyenland, KY 94975",1,0,Failed,Online
6,Ashley Rodriguez,robert47@hotmail.com,0412321372,101,Rag Type A,2024-08-09,20,5.0,100.0,Cash,"363 Jamie Union, South Jacobmouth, VT 39768",5,15%,Failed,In-store
7,Courtney Maxwell,oshields@zuniga.com,(805)454-2153,104,Rag Type D,2024-01-02,20,4.75,95.0,PayPal,"10048 Michael Light Suite 319, Aprilburgh, MN 62279",3,0,Failed,In-store
8,Kenneth Santiago,karen35@gmail.com,+1-916-832-8395,102,Rag Type B,2024-02-07,19,3.5,66.5,PayPal,"61924 Rhonda Light Suite 603, Stacymouth, NH 33187",1,5%,Successful,In-store
9,Michele Watson,wrightdawn@little-stevens.com,846-977-0406x0581,105,Rag Type E,2024-05-31,9,7.0,63.0,Cash,"67434 Amanda Crossing Apt. 787, East Sherry, WY 08132",1,5%,Successful,Online
10,Anthony Hudson,nicole62@young-jones.org,(029)984-1814x944,102,Rag Type B,2024-06-17,10,3.5,35.0,PayPal,"553 Joseph Club Suite 629, North Kristenborough, VA 79147",5,5%,Failed,In-store
11,Tiffany Ford,jaimesmith@yahoo.com,340.546.7716x6737,103,Rag Type C,2024-01-04,15,6.25,93.75,Credit Card,"98921 Young Trafficway Apt. 382, Edwardbury, WY 00625",4,15%,Failed,In-store
12,Ashley Baker,watkinspamela@acosta-cox.com,398-654-2150x0900,101,Rag Type A,2024-09-27,11,5.0,55.0,PayPal,"517 Bradley Trail, South Tiffany, KY 19407",4,15%,Failed,Online
13,Rachel Adams,amy14@hotmail.com,705.873.3103,105,Rag Type E,2024-01-11,17,7.0,119.0,Cash,"694 Gregory Ports, Wilsonfort, MT 53261",3,5%,Successful,In-store
14,Jennifer Rivas,michaelevans@henderson-peterson.com,(501)246-0457x7643,102,Rag Type B,2024-06-21,8,3.5,28.0,Credit Card,"17499 Vernon Extensions Apt. 733, Elizabethmouth, WY 98367",3,5%,Failed,Online
15,Charles Anthony,courtney70@hotmail.com,+1-255-444-8110x9185,101,Rag Type A,2024-02-22,15,5.0,75.0,Cash,"Unit 8813 Box 4513, DPO AA 71494",2,0,Failed,Online
16,Joseph Thompson,sfields@howard.com,(390)404-3532x04726,104,Rag Type D,2024-08-07,3,4.75,14.25,Credit Card,"25267 Christopher Union, Kingfurt, AK 56395",1,15%,Failed,In-store
17,Guy Bruce,jamierussell@gmail.com,+1-265-304-9291x44285,102,Rag Type B,2024-09-07,15,3.5,52.5,Credit Card,"57215 Robin Hills Apt. 991, Vazquezchester, WA 69663",1,15%,Failed,Online
18,Benjamin Shea,triciasingh@yahoo.com,(330)285-7027x288,102,Rag Type B,2024-06-13,2,3.5,7.0,Cash,"60442 Jeff Skyway Suite 244, Barronbury, VA 12660",5,0,Failed,Online
19,Charles Lucas,richardsimmons@garrett-lee.biz,001-011-440-7906x086,101,Rag Type A,2024-08-02,2,5.0,10.0,PayPal,"0367 Dalton Mission Suite 151, North Jessicashire, IN 78138",3,5%,Failed,Online
20,Sherry Prince,matthewblanchard@edwards-edwards.com,516.770.9367,105,Rag Type E,2024-02-06,19,7.0,133.0,Cash,"1807 Alejandra Loaf, Smithbury, PA 22919",3,15%,Successful,Online
21,Brian Goodman,hillanthony@gmail.com,879-243-0916x6780,104,Rag Type D,2024-05-21,12,4.75,57.0,PayPal,"1581 Cesar Harbor, Danielleville, MA 91055",5,10%,Successful,In-store
22,Martin Duran,cobbjody@welch-thompson.info,564-552-6340x689,103,Rag Type C,2024-01-09,18,6.25,112.5,PayPal,"9857 Cassandra Plaza, Lake Christine, MA 38628",1,15%,Failed,In-store
23,Anthony Alvarez,roberthenderson@hotmail.com,+1-545-343-9482x944,102,Rag Type B,2024-01-26,19,3.5,66.5,PayPal,"6932 Wood Groves, Lake Eugene, NM 15072",3,10%,Failed,In-store
24,Linda Donaldson,goodwinchelsea@gmail.com,598.668.6042x500,103,Rag Type C,2024-02-07,17,6.25,106.25,PayPal,"38250 Alyssa Motorway Suite 874, South Melanie, CO 58328",5,15%,Failed,In-store
25,Shannon Allen,franciscoreyes@hotmail.com,+1-011-982-7255x872,103,Rag Type C,2024-09-23,7,6.25,43.75,Credit Card,"8009 Amy Cape, Carterbury, NY 98118",3,5%,Successful,Online
26,Andrew Schroeder,wolfemichael@torres.biz,023-689-1567,103,Rag Type C,2024-02-24,8,6.25,50.0,Cash,"USCGC Tyler, FPO AA 34708",3,5%,Failed,In-store
27,Mary Brooks,jonathanmeyer@yahoo.com,654.582.2681x695,101,Rag Type A,2024-08-12,13,5.0,65.0,Credit Card,"Unit 6539 Box 9707, DPO AP 75389",5,5%,Failed,Online
28,Kevin Lee,mramsey@lopez.com,799-022-7412,105,Rag Type E,2024-02-01,20,7.0,140.0,Cash,"9948 Guerra Islands, Griffinburgh, MD 53143",1,0,Failed,In-store
29,John Sanchez,mitchellbryan@gmail.com,001-473-198-8871x09355,102,Rag Type B,2024-07-01,20,3.5,70.0,Credit Card,"42670 Joe Points Apt. 930, Lake Sabrina, MO 45110",5,0,Failed,Online
30,Elizabeth Allen,qharrington@yahoo.com,001-935-540-3636x71437,103,Rag Type C,2024-08-25,7,6.25,43.75,PayPal,"89464 Kelsey Plaza, South Marvinville, MI 92758",3,5%,Successful,Online
31,David Porter,watkinstimothy@hotmail.com,+1-594-154-0163x891,104,Rag Type D,2024-02-12,6,4.75,28.5,Cash,"10938 Brian Villages, Lake Jennifer, WA 35709",4,15%,Successful,In-store
32,Susan Jackson,gschwartz@hunter.biz,9111743214,102,Rag Type B,2024-05-25,9,3.5,31.5,PayPal,"7864 Avila Courts Suite 041, Lewisshire, KY 70239",4,10%,Failed,Online
33,William Bruce,trevinomichael@morris-white.net,+1-563-221-6443x5900,103,Rag Type C,2024-07-23,19,6.25,118.75,PayPal,"Unit 8144 Box 7717, DPO AA 76386",5,0,Failed,Online
34,Steven Fox,lbrown@yahoo.com,(189)554-9552x7398,101,Rag Type A,2024-02-07,8,5.0,40.0,PayPal,"690 Cook Prairie Apt. 165, North Stephanieton, MO 64724",1,10%,Successful,Online
35,Michelle Fischer,lindafarley@robinson.info,652.313.2598,105,Rag Type E,2024-06-19,15,7.0,105.0,Credit Card,"694 Amy Heights, Johnsonburgh, CO 29825",3,5%,Failed,In-store
36,Mark Murphy,harriscynthia@gmail.com,455-685-9568,101,Rag Type A,2024-08-31,14,5.0,70.0,PayPal,"2579 Erica Prairie Apt. 350, South Ryanfurt, AK 70181",1,10%,Failed,Online
37,Justin Oconnor,toddramirez@hotmail.com,+1-646-568-6827,101,Rag Type A,2024-08-10,14,5.0,70.0,Cash,"030 Donna Stravenue, Stephanietown, WI 70451",4,5%,Failed,In-store
38,Matthew Roberts,michelle05@yahoo.com,(585)665-8814,105,Rag Type E,2024-07-16,3,7.0,21.0,Cash,"949 Monica Estate, Lindahaven, MS 50670",2,5%,Failed,Online
39,Jackie James,andrew05@yahoo.com,001-986-628-8057,104,Rag Type D,2024-08-01,9,4.75,42.75,Cash,"59698 Campos Harbor Apt. 778, Marissashire, KY 83243",4,15%,Successful,Online
40,Erin Morgan,nmorris@morgan-smith.com,(657)025-1608,102,Rag Type B,2024-05-28,16,3.5,56.0,Credit Card,"62597 Gardner Ports, Jamieshire, ME 91352",4,0,Successful,Online
41,Haley Henderson,melissamorrison@lopez.biz,(195)069-6033x7286,104,Rag Type D,2024-05-06,1,4.75,4.75,PayPal,"20400 Williams Trail, West Denise, LA 18099",2,15%,Successful,In-store
42,Cassandra Washington,matthewreynolds@bowen-greene.com,(339)226-9818x63879,101,Rag Type A,2024-01-08,4,5.0,20.0,Credit Card,"056 Michael Estates, Michelleville, MT 67432",4,5%,Successful,In-store
43,Dana Jennings,trevor95@allen.info,001-635-983-9853x36369,105,Rag Type E,2024-07-14,8,7.0,56.0,Cash,"46351 Wise Shoals Apt. 769, Lake Richardshire, SD 08794",4,15%,Failed,Online
44,Dr. Christian Bentley,tina51@palmer.net,751-106-6631x26829,104,Rag Type D,2024-01-11,19,4.75,90.25,Cash,"1438 Brenda Estates Suite 280, Peterton, UT 34263",3,5%,Failed,In-store
45,Mike Goodman,tjenkins@gmail.com,(873)748-5871,103,Rag Type C,2024-04-04,15,6.25,93.75,Credit Card,"001 Reed Squares Suite 499, Michaelside, DE 95982",3,0,Successful,In-store
46,Daisy Alvarado,danamartinez@yahoo.com,943-177-1194x42742,102,Rag Type B,2024-09-05,8,3.5,28.0,PayPal,"1921 Gonzales Lodge, Michaelbury, WA 33976",2,5%,Successful,In-store
47,Ashley Melton,rosetran@colon.com,(895)340-0553x5020,102,Rag Type B,2024-01-25,17,3.5,59.5,Cash,"24771 Tina View, Lauraborough, ME 33240",3,0,Successful,In-store
48,Penny Guerra,boydricky@hotmail.com,980.205.2652x46497,105,Rag Type E,2024-06-15,10,7.0,70.0,Credit Card,"156 Lane Circles, Mcmahonfurt, FL 55831",5,10%,Failed,Online
49,Patrick Smith,susanmiller@hotmail.com,+1-592-300-0650,102,Rag Type B,2024-07-22,8,3.5,28.0,PayPal,"84809 Christian Lane Suite 756, East Paulmouth, MS 64179",3,0,Failed,In-store
50,Bobby Anderson,zdalton@stone-pruitt.com,001-451-604-3319x09782,103,Rag Type C,2024-09-11,19,6.25,118.75,Cash,"48901 Jennifer Crossing Suite 846, West Justin, GA 57429",1,5%,Successful,In-store
51,Jennifer Delgado,loganjohn@davis-kim.com,700.228.7156x2948,104,Rag Type D,2024-01-22,17,4.75,80.75,PayPal,"612 Harry Ports, Whiteland, NJ 76721",1,0,Failed,Online
52,Sean Landry,clinton35@smith-boyd.org,196.244.1677x71320,104,Rag Type D,2024-06-28,7,4.75,33.25,PayPal,"39975 Nathan Estates, North Samantha, WV 62382",5,0,Successful,In-store
53,Amanda Benton,clarkechristopher@edwards.org,+1-447-190-2944x7097,102,Rag Type B,2024-09-16,12,3.5,42.0,Cash,"552 Soto Drive, Westside, TX 76060",4,15%,Failed,In-store
54,Jessica Mitchell,williamlove@lopez.com,001-961-932-4374,101,Rag Type A,2024-08-08,14,5.0,70.0,Credit Card,"01091 Perkins Stream Suite 730, New Johnport, WY 62256",5,15%,Failed,Online
55,Karen Allen,isaunders@hotmail.com,0282566358,105,Rag Type E,2024-01-13,17,7.0,119.0,Cash,"79241 John Ramp, West Kathychester, OR 91093",2,0,Failed,In-store
56,Erin Clarke,vmoore@gmail.com,345.806.0937,103,Rag Type C,2024-01-08,4,6.25,25.0,Credit Card,"1015 Terrence Grove, North Sarah, MO 59050",1,10%,Failed,Online
57,Adam Brown,eric44@clay.com,889-044-6340x618,104,Rag Type D,2024-01-19,20,4.75,95.0,Cash,"13543 Leonard Forges, North Joshua, AR 24016",3,10%,Successful,In-store
58,Abigail Lee,moralesadriana@gmail.com,(545)006-4165,102,Rag Type B,2024-03-09,20,3.5,70.0,PayPal,"200 Newman Cove Suite 301, North Richardshire, WA 28359",2,0,Successful,In-store
59,Jean Miller,xbryant@torres.info,144-530-4481x86985,102,Rag Type B,2024-08-29,15,3.5,52.5,Cash,"97246 Elizabeth Road, New Meghanside, CO 60035",3,5%,Failed,In-store
60,Mark Morris,aibarra@robbins.info,4993896791,101,Rag Type A,2024-04-10,14,5.0,70.0,Cash,"89029 Pratt Highway, Port Bobby, LA 88981",5,15%,Successful,Online
61,Joanne Travis,opadilla@lynch.com,(341)247-1710,103,Rag Type C,2024-05-14,18,6.25,112.5,Credit Card,"784 Samantha Landing, Erinport, NY 50314",3,5%,Successful,In-store
62,William Avery,patrickturner@gmail.com,+1-448-359-2096x0932,104,Rag Type D,2024-07-10,9,4.75,42.75,PayPal,"135 Emily Keys Suite 720, East Abigailmouth, MN 47635",5,10%,Successful,In-store
63,Jennifer Raymond,martin66@richardson-ellis.com,6188060003,104,Rag Type D,2024-02-18,18,4.75,85.5,Cash,"74615 Andrew Center, South Elizabethchester, MS 49428",5,10%,Successful,Online
64,Jason Walsh,michaelperry@brown.info,(208)993-8839x54782,105,Rag Type E,2024-04-12,2,7.0,14.0,Cash,"520 Martinez Pass, West Tracy, VA 23312",5,10%,Failed,In-store
65,Laura Blackburn,kathryn37@gmail.com,884-094-5961x8901,105,Rag Type E,2024-02-22,5,7.0,35.0,PayPal,"8165 Phelps Key Apt. 992, Harrisonfort, NJ 75547",2,10%,Successful,Online
66,Joanna Woods,fcole@gmail.com,622-456-1285x12116,102,Rag Type B,2024-05-11,1,3.5,3.5,Credit Card,"76404 Hubbard Gateway Suite 754, New Carol, DC 22953",1,15%,Successful,In-store
67,David Roberts,stephaniedawson@medina.biz,466.800.1502x908,105,Rag Type E,2024-04-20,18,7.0,126.0,Credit Card,"PSC 2264, Box 2930, APO AP 70280",1,0,Failed,Online
68,Stephen Melton,hillkaren@myers.info,287.036.1899x9632,104,Rag Type D,2024-08-16,18,4.75,85.5,PayPal,"001 Ashley Parks Suite 684, New Isaacburgh, VA 15359",4,10%,Successful,In-store
69,Jeremy Rodriguez,michael56@hotmail.com,362-340-8966x26080,104,Rag Type D,2024-02-13,19,4.75,90.25,PayPal,"6522 Jeffery Station Apt. 666, Martinberg, NM 85754",4,10%,Failed,Online
70,John Thompson,benitezmelissa@hotmail.com,939.982.5672x20082,102,Rag Type B,2024-07-17,18,3.5,63.0,PayPal,"51606 Margaret Highway, Erikmouth, AZ 34874",2,10%,Failed,Online
71,Alan Mann,gregory82@hotmail.com,(663)270-2936x029,105,Rag Type E,2024-02-05,16,7.0,112.0,Credit Card,"142 Katherine Pass, Nicoleport, AL 07762",5,10%,Successful,Online
72,Keith Johnson,kscott@rose-cantrell.com,836.895.0967x54405,104,Rag Type D,2024-08-22,1,4.75,4.75,Credit Card,"260 Austin Overpass Apt. 523, East Christopher, CT 05581",1,15%,Failed,Online
73,Carolyn Ramirez,nicholsjoan@houston.com,758-522-0837,102,Rag Type B,2024-04-23,9,3.5,31.5,Credit Card,"8648 Diaz Fall Suite 002, East Brittanyside, IN 33520",5,0,Successful,Online
74,Stephen Franco,quinnnicole@burnett-scott.net,481.037.5044,104,Rag Type D,2024-03-09,15,4.75,71.25,PayPal,"369 Cynthia Harbor Apt. 471, North Sabrina, NC 75181",3,0,Failed,In-store
75,Amy Mcneil,serranojoe@horn.org,+1-824-762-9874x71080,105,Rag Type E,2024-01-24,16,7.0,112.0,PayPal,"Unit 2793 Box 2063, DPO AE 64797",3,10%,Failed,In-store
76,Alexandra Chapman,hcampbell@morales.com,001-155-797-0846x1721,102,Rag Type B,2024-04-19,15,3.5,52.5,Credit Card,"6336 Obrien Rapids, Annatown, IL 73359",3,10%,Failed,In-store
77,Jared Mcdonald,thomas30@gmail.com,(434)915-9454,104,Rag Type D,2024-03-11,20,4.75,95.0,PayPal,"45146 Don Lake Suite 945, New Elizabethmouth, SC 71401",5,0,Failed,In-store
78,Jeremy Young,pbutler@hotmail.com,5666971564,105,Rag Type E,2024-05-23,19,7.0,133.0,Credit Card,"8547 Jessica Path, East Jessicabury, WI 39566",5,5%,Failed,Online
79,Arthur Williams,deborahsmith@browning.info,725-008-7738,102,Rag Type B,2024-01-22,8,3.5,28.0,Cash,"PSC 0576, Box 4126, APO AP 88575",4,0,Successful,In-store
80,Anthony Reese,keithperez@yahoo.com,+1-475-657-6616,102,Rag Type B,2024-04-17,2,3.5,7.0,Credit Card,"162 Melissa Trafficway, Port Derek, MO 77044",3,10%,Successful,In-store
81,Jasmine Jimenez,cynthiasanders@gomez.com,232-476-8799,104,Rag Type D,2024-02-10,4,4.75,19.0,PayPal,"PSC 4878, Box 9444, APO AE 97510",1,0,Successful,Online
82,Stephanie Davis,fmckinney@hotmail.com,577-795-7925,103,Rag Type C,2024-02-20,2,6.25,12.5,PayPal,"81007 Chelsea Field, Hunterport, SC 97313",2,0,Failed,In-store
83,George Todd,charlesariana@hicks.info,987.366.4337,102,Rag Type B,2024-04-26,5,3.5,17.5,PayPal,"70854 Jessica Highway, Johnchester, ID 42537",2,15%,Successful,In-store
84,Adam Wright,ggriffin@escobar.com,(361)129-7369x612,104,Rag Type D,2024-02-15,11,4.75,52.25,Cash,"75280 Johnson Way Suite 484, East Christine, WY 76571",2,5%,Failed,In-store
85,Jessica Alexander,olivia92@hart.com,011-280-7018x5617,105,Rag Type E,2024-09-08,8,7.0,56.0,Cash,"93634 Christian Prairie Apt. 160, Charlesfort, ND 25067",4,5%,Successful,In-store
86,Patricia Barker,allison37@harding-cox.com,+1-073-199-7173x8772,103,Rag Type C,2024-07-12,19,6.25,118.75,PayPal,"286 Oliver Route Suite 408, North Timothyville, UT 06388",5,0,Successful,In-store
87,Stephanie Price,fisherwhitney@gmail.com,+1-934-735-4976x2990,103,Rag Type C,2024-04-05,4,6.25,25.0,PayPal,"PSC 1889, Box 0618, APO AA 19988",2,0,Successful,In-store
88,Michael Kaiser,rileytanya@smith.info,+1-633-434-6356,101,Rag Type A,2024-06-14,17,5.0,85.0,PayPal,"862 Castro Course, Lake Leslie, KY 55209",2,0,Failed,In-store
89,Brian Harper,anthonyruiz@williams.com,(298)670-8611x40142,102,Rag Type B,2024-08-25,1,3.5,3.5,Credit Card,"20226 Alexander Points Apt. 307, Donaldport, MA 46191",1,15%,Successful,Online
90,Alexandra Stout,jeffreymason@gmail.com,(628)592-0243,102,Rag Type B,2024-02-25,3,3.5,10.5,Cash,"5328 Pamela Extension, South Johnchester, GA 64539",1,5%,Successful,Online
91,Hannah Thompson,robertoconnor@hotmail.com,023-221-2152x1807,103,Rag Type C,2024-07-06,11,6.25,68.75,PayPal,"716 Ayala Turnpike Apt. 580, New Lisa, SC 91203",4,0,Failed,In-store
92,Ashley Hansen,reneebailey@finley.org,001-562-372-1248,104,Rag Type D,2024-07-14,19,4.75,90.25,Cash,"513 Cameron Trail Suite 795, Martinezhaven, WY 18462",1,15%,Successful,In-store
93,Anna Winters,nmathis@yahoo.com,+1-557-630-1533x287,104,Rag Type D,2024-03-31,11,4.75,52.25,PayPal,"45589 Perez Junction Apt. 292, Stacymouth, GA 63957",3,15%,Failed,Online
94,Christopher Woods,msilva@hotmail.com,+1-579-372-5198x2505,105,Rag Type E,2024-04-04,7,7.0,49.0,Cash,"3805 Kenneth Corners Apt. 229, Emilybury, VA 27315",2,10%,Successful,Online
95,Melinda Rojas,espinozajoe@yahoo.com,(875)757-5154x346,102,Rag Type B,2024-04-14,7,3.5,24.5,Credit Card,"68046 Joseph Lodge, East Philiphaven, OK 58254",1,0,Failed,In-store
96,Sabrina Roman MD,aaron54@prince-maldonado.com,020.978.7280x27025,103,Rag Type C,2024-07-13,19,6.25,118.75,Cash,"140 Watson Forest Suite 836, East Williamville, SC 83763",4,10%,Successful,Online
97,Wendy Woodard,sarahmartin@yahoo.com,759-994-1951,101,Rag Type A,2024-07-24,15,5.0,75.0,Cash,"303 Cooper Pine, Higginsfort, WV 20783",3,5%,Successful,Online
98,Jill Mclaughlin,frasmussen@hotmail.com,2845069538,103,Rag Type C,2024-07-14,5,6.25,31.25,PayPal,"695 Rebecca Shoal Suite 717, Guzmanmouth, WY 77966",5,0,Successful,Online
99,James Payne,william72@gmail.com,(934)424-1547,102,Rag Type B,2024-02-19,1,3.5,3.5,PayPal,"33196 Jared Via Apt. 614, Colinmouth, ME 54322",5,0,Successful,In-store
100,Mr. James Benitez,william49@kim-johnson.com,0615663437,104,Rag Type D,2024-01-17,12,4.75,57.0,PayPal,"PSC 2713, Box 6247, APO AE 68972",4,5%,Successful,Online
101,Katie Foster,stacywilliams@gmail.com,743.702.5915,104,Rag Type D,2024-04-01,3,4.75,14.25,Credit Card,"18423 Gordon Walk Apt. 920, East Jeremy, NJ 19216",3,5%,Failed,Online
102,Tammy Callahan,gacosta@gmail.com,4928821420,103,Rag Type C,2024-06-27,2,6.25,12.5,Credit Card,"1305 Rodriguez Land, South Kristi, KY 78509",4,5%,Failed,Online
103,Victoria Kent,tiffanywalker@hotmail.com,+1-280-778-5032x62004,102,Rag Type B,2024-02-05,19,3.5,66.5,PayPal,"98042 Bridget Summit Apt. 252, Bethfort, MN 89394",1,10%,Failed,Online
104,Juan Roach,dmelendez@yahoo.com,+1-828-281-1295x77729,102,Rag Type B,2024-01-04,20,3.5,70.0,Credit Card,"PSC 0942, Box 6432, APO AP 86166",1,5%,Failed,Online
105,Samuel Aguilar,bberry@foster.com,697.560.4172x37445,103,Rag Type C,2024-02-10,5,6.25,31.25,Credit Card,"1988 Nunez Mission, Port Davidton, AK 72421",4,5%,Failed,Online
106,Susan Smith,owilcox@castro.com,205-111-8267x474,105,Rag Type E,2024-05-15,5,7.0,35.0,Credit Card,"89636 Anderson Well Apt. 332, Marvinbury, MD 15342",4,5%,Successful,Online
107,Angela Franklin,liulauren@baldwin.biz,(655)675-8046,102,Rag Type B,2024-02-22,12,3.5,42.0,PayPal,"3151 Scott Motorway Suite 392, Gardnershire, KS 68738",3,0,Failed,Online
108,Lisa David,paula62@crawford.info,279.759.6540x01726,104,Rag Type D,2024-09-10,8,4.75,38.0,PayPal,"1450 Stanley Turnpike Apt. 968, Leonton, AZ 96805",1,0,Successful,Online
109,Benjamin Gordon,swilson@martinez.com,(454)163-3016,104,Rag Type D,2024-02-10,17,4.75,80.75,Credit Card,"15513 Kennedy Via, Margaretview, NY 07291",5,15%,Failed,Online
110,Bridget Hernandez,jamesthompson@yahoo.com,+1-752-551-2353x6718,101,Rag Type A,2024-05-21,6,5.0,30.0,Credit Card,"5860 Andrew Crescent, Cathyport, OR 60111",2,0,Successful,Online
111,Darrell Ramsey,tonyafoster@hotmail.com,987.570.8364x68128,104,Rag Type D,2024-04-18,12,4.75,57.0,PayPal,"130 Michael Place Suite 294, South Meredith, TN 05154",4,5%,Successful,Online
112,Marcus Thomas Jr.,zmarsh@yahoo.com,+1-025-133-8700x553,105,Rag Type E,2024-05-18,7,7.0,49.0,PayPal,"15969 Emily Roads, South William, OR 92973",2,0,Successful,In-store
113,Melissa Rivera,cruzbrian@hotmail.com,189.526.0567,102,Rag Type B,2024-06-24,8,3.5,28.0,Credit Card,"23947 Ashlee Circles Suite 980, Diazview, KY 48398",4,0,Successful,Online
114,Bridget Benton,ghaley@gmail.com,149-589-5532,102,Rag Type B,2024-07-28,10,3.5,35.0,Cash,"USNS Ross, FPO AE 02482",4,10%,Successful,In-store
115,Christine Fitzgerald,hpitts@gmail.com,001-673-233-2098x333,105,Rag Type E,2024-05-24,13,7.0,91.0,Credit Card,"0117 Sarah Ramp Apt. 152, Vasquezbury, NM 47509",1,0,Failed,In-store
116,Ashley Paul,zsmith@yahoo.com,(348)600-8261,105,Rag Type E,2024-08-29,4,7.0,28.0,PayPal,"PSC 3365, Box 2188, APO AP 91830",2,10%,Failed,Online
117,Kevin Bush,qbyrd@gmail.com,+1-082-337-7711x588,104,Rag Type D,2024-09-08,5,4.75,23.75,Cash,"PSC 6364, Box 8926, APO AP 18634",1,10%,Failed,Online
118,Robert Morgan,stephen34@harris-thomas.com,889-173-2341,105,Rag Type E,2024-02-06,3,7.0,21.0,PayPal,"20785 Davis Harbor Apt. 669, Reyesstad, ME 14760",1,10%,Successful,In-store
119,Jeffrey White,bgonzalez@yahoo.com,001-550-637-7797x85010,103,Rag Type C,2024-06-04,16,6.25,100.0,Cash,"76252 Jennifer Expressway, Lake Samantha, ME 58085",2,15%,Successful,Online
120,Mr. Erik Hale,david53@gmail.com,+1-339-180-4033,101,Rag Type A,2024-01-18,18,5.0,90.0,Credit Card,"Unit 7439 Box 8066, DPO AE 53630",2,5%,Failed,In-store
121,Christopher Haynes,klewis@gmail.com,353-315-2922,103,Rag Type C,2024-06-28,15,6.25,93.75,PayPal,"857 King Locks, Cruzberg, NV 82212",1,5%,Failed,Online
122,Sharon Bullock,ddavis@yahoo.com,(427)705-2382x752,105,Rag Type E,2024-08-26,11,7.0,77.0,Cash,"6744 Hernandez Plaza, South Amanda, DE 53475",1,15%,Successful,In-store
123,Jay Summers,michael80@gmail.com,001-169-708-0071,105,Rag Type E,2024-01-07,15,7.0,105.0,Credit Card,"22081 Michelle Crescent Apt. 762, Perryville, MD 79549",2,10%,Successful,In-store
124,Kevin Owen,achen@smith-morris.com,1227078082,105,Rag Type E,2024-04-02,11,7.0,77.0,PayPal,"002 Allen Hills, West Wanda, WV 75634",4,0,Successful,Online
125,Nathan Drake,cdaniels@hotmail.com,001-098-799-8745x5338,105,Rag Type E,2024-01-02,4,7.0,28.0,PayPal,"Unit 0497 Box 1589, DPO AE 47098",4,10%,Failed,In-store
126,Kayla Walker,codysanchez@gmail.com,665.657.4869x56765,102,Rag Type B,2024-01-07,9,3.5,31.5,Credit Card,"0642 David Forks, Mooreborough, NV 08677",4,15%,Failed,Online
127,Brian Meadows,blankenshipkimberly@meyer-bowers.com,090.605.4126x95014,101,Rag Type A,2024-08-28,12,5.0,60.0,Credit Card,"30127 Jenkins Plains Suite 563, Lake Gabriel, MT 42965",1,5%,Failed,In-store
128,Margaret Rice,mcclurealicia@hotmail.com,(983)375-3780x966,102,Rag Type B,2024-01-12,17,3.5,59.5,PayPal,"146 Jody Heights Suite 491, Christopherfurt, VT 60118",1,5%,Failed,Online
129,Robin Hendrix,andersonstephanie@hotmail.com,(532)465-5584,101,Rag Type A,2024-01-17,17,5.0,85.0,PayPal,"7903 Shaffer Villages, West Daisyhaven, WA 93131",5,15%,Failed,Online
130,Todd Rodriguez,jacksongreg@hotmail.com,(252)920-1315x06900,105,Rag Type E,2024-05-09,12,7.0,84.0,PayPal,"069 Adam Drive Suite 191, Duncanfurt, DC 19271",2,10%,Failed,In-store
131,Russell Moore,jessica25@gmail.com,282.761.8824,103,Rag Type C,2024-09-06,14,6.25,87.5,PayPal,"7166 Susan Crossroad Apt. 861, Bobbyland, OR 02301",5,15%,Successful,In-store
132,Sara Thompson,noahallen@smith-williams.com,(092)131-2139x1090,105,Rag Type E,2024-09-10,4,7.0,28.0,Credit Card,"8027 Bennett Fords, Whiteland, LA 81642",3,0,Failed,Online
133,Theodore Smith,griffinerin@dixon-smith.net,905.983.5338x211,103,Rag Type C,2024-06-14,3,6.25,18.75,Cash,"380 Dana Plaza Suite 025, Leonardland, CA 26626",2,15%,Failed,In-store
134,Blake Vance,masonjohn@yahoo.com,1997092356,102,Rag Type B,2024-01-30,4,3.5,14.0,Credit Card,"1569 Johnston Bridge, Port Benjaminview, NH 06620",4,10%,Failed,In-store
135,William Zamora,josephrobinson@yahoo.com,869.906.8843,105,Rag Type E,2024-06-03,14,7.0,98.0,Cash,"PSC 3952, Box 1107, APO AA 49542",5,15%,Failed,Online
136,Michael Jones,dgarrett@hotmail.com,241.107.7959x7743,105,Rag Type E,2024-02-27,6,7.0,42.0,Cash,"98024 Paul Mills Apt. 970, Hartchester, VA 90984",4,0,Successful,In-store
137,Randall Foster,christopher61@gray.com,425-047-7812,102,Rag Type B,2024-03-27,15,3.5,52.5,Credit Card,"89466 Brittney Crossing, West John, ND 45269",1,0,Successful,Online
138,Mr. John Scott,rioschristopher@cole.com,184-714-6812x71635,101,Rag Type A,2024-07-11,14,5.0,70.0,Credit Card,"945 Kathleen Pass Apt. 450, West James, NH 30165",4,5%,Failed,In-store
139,Cassandra Livingston,cynthia60@hotmail.com,001-071-967-6403x46811,104,Rag Type D,2024-02-13,19,4.75,90.25,PayPal,"463 Gregory Overpass, East Johnmouth, NH 61005",2,5%,Failed,Online
140,Erika Ellis,hhawkins@gutierrez.com,+1-588-675-6239x92150,104,Rag Type D,2024-02-27,15,4.75,71.25,Cash,"12028 Campbell Skyway, Cynthiaview, OR 34972",2,0,Failed,Online
141,John Young,martincindy@gmail.com,3454871846,104,Rag Type D,2024-05-17,1,4.75,4.75,Cash,"Unit 5987 Box 3404, DPO AE 31323",4,10%,Successful,Online
142,Shannon Lopez,hortonkimberly@hotmail.com,001-408-824-6617,104,Rag Type D,2024-06-27,16,4.75,76.0,PayPal,"PSC 0556, Box 8461, APO AE 53177",3,15%,Failed,In-store
143,Erika Johnson,timothy56@gmail.com,(715)399-9728,101,Rag Type A,2024-05-15,20,5.0,100.0,Cash,"13714 Darlene Cliff Apt. 035, Thomasborough, AZ 70254",3,5%,Failed,In-store
144,Chad Barnett,heatherjackson@yahoo.com,465.698.6543x54243,104,Rag Type D,2024-02-29,13,4.75,61.75,Credit Card,"529 Kara Field Suite 513, Williamstad, MO 95805",5,10%,Failed,In-store
145,Shelby Anderson,esingh@hotmail.com,001-478-582-5324x64754,103,Rag Type C,2024-07-15,6,6.25,37.5,Credit Card,"77443 Barry Ridges, Grayfurt, TX 40526",4,15%,Successful,Online
146,Gregory Scott,abrock@cameron.info,+1-707-811-5126x403,102,Rag Type B,2024-02-15,5,3.5,17.5,Credit Card,"0781 Amy Extensions, Wilsonfort, MT 54816",3,5%,Failed,Online
147,Lacey Hicks,thomas39@hotmail.com,891-478-0329,103,Rag Type C,2024-07-01,15,6.25,93.75,Credit Card,"85926 Mejia Loaf Suite 598, Harrisberg, MN 94520",1,15%,Failed,Online
148,Sabrina Washington,scott94@neal.info,001-918-906-0306,103,Rag Type C,2024-09-17,2,6.25,12.5,PayPal,"80376 Jenkins Ford Apt. 436, East Jeremyburgh, MA 84679",1,10%,Failed,Online
149,Raven Fernandez,donaldbrooks@gmail.com,818-106-3726x7485,105,Rag Type E,2024-02-19,4,7.0,28.0,Credit Card,"87654 Bryan Row Suite 252, Melendezburgh, TX 93759",4,5%,Failed,In-store
150,Alexis Steele,gutierrezcolleen@yahoo.com,202.830.5484x82801,103,Rag Type C,2024-07-09,8,6.25,50.0,PayPal,"49665 Michael Lane, New Cynthiachester, GA 09118",5,0,Failed,Online
"""

In [4]:
from dotenv import load_dotenv
import os
load_dotenv()
google_gemini_api = os.getenv("GOOGLE_API_KEY")
pinecone_api=os.getenv("PINECONE_API_KEY")

In [5]:
from io import StringIO
def read_csv_from_string(data):
    return pd.read_csv(StringIO(data))

# Read the CSV
df = read_csv_from_string(data)

In [6]:
from langchain_google_genai import ChatGoogleGenerativeAI

/Users/anuragsingh/Documents/GitHub/interTask/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
google_gemini_api = os.getenv("GOOGLE_API_KEY")
llm_model = ChatGoogleGenerativeAI(model="gemini-1.5-flash", google_api_key=google_gemini_api)

In [8]:
!pip install langchain_experimental

I0000 00:00:1728213042.009360  196726 fork_posix.cc:77] Other threads are currently calling into gRPC, skipping fork() handlers


  Using cached langchain_experimental-0.3.2-py3-none-any.whl.metadata (1.7 kB)
Using cached langchain_experimental-0.3.2-py3-none-any.whl (208 kB)


In [9]:
!pip install tabulate

I0000 00:00:1728213045.096014  196726 fork_posix.cc:77] Other threads are currently calling into gRPC, skipping fork() handlers


  Using cached tabulate-0.9.0-py3-none-any.whl.metadata (34 kB)
Using cached tabulate-0.9.0-py3-none-any.whl (35 kB)


In [10]:
from langchain_experimental.agents import create_csv_agent
file_path = 'customer_data.csv'
df.to_csv(file_path, index=False)
agent = create_csv_agent(llm_model, file_path, verbose=True, allow_dangerous_code=True)

In [12]:
from io import StringIO
from contextlib import redirect_stdout
def capture_verbose_log(agent, query):
    f = StringIO()
    
    with redirect_stdout(f):
        try:
            result = agent.run(query)
        except Exception as e:
            result = f"Error occurred: {e}"

    verbose_log = f.getvalue()
    return result, verbose_log

In [24]:
query = "What is the average feedback score for Rag Type A?"
response, log_output = capture_verbose_log(agent,query)

In [25]:
print(response)

3.210526315789474 
6315789474


In [26]:
print(log_output)



> Entering new AgentExecutor chain...
Thought: I need to find all the rows in the dataframe that have a Product Name of "Rag Type A" and then calculate the average of the Feedback Score column for those rows.
Action: python_repl_ast
Action Input: df[df['Product Name'] == 'Rag Type A']['Feedback Score'].mean()3.210526315789474Thought: I now know the final answer
Final Answer: 3.210526315789474 
6315789474 


> Finished chain.



In [27]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=300,
    chunk_overlap=60
)

In [28]:
text_chunks = text_splitter.split_text(log_output)

In [29]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings
embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")

In [19]:
!pip install pinecone

I0000 00:00:1728213073.816908  196726 fork_posix.cc:77] Other threads are currently calling into gRPC, skipping fork() handlers


  Using cached pinecone-5.3.1-py3-none-any.whl.metadata (19 kB)
  Using cached pinecone_plugin_inference-1.1.0-py3-none-any.whl.metadata (2.2 kB)
  Using cached pinecone_plugin_interface-0.0.7-py3-none-any.whl.metadata (1.2 kB)
Using cached pinecone-5.3.1-py3-none-any.whl (419 kB)
Using cached pinecone_plugin_inference-1.1.0-py3-none-any.whl (85 kB)
Using cached pinecone_plugin_interface-0.0.7-py3-none-any.whl (6.2 kB)


In [30]:
#initializing the pinecone db
from pinecone import Pinecone
pc = Pinecone(api_key=pinecone_api)
index = pc.Index("mr-analyzer")

In [31]:
from langchain.vectorstores import Pinecone as PC
docs_chunks = [t for t in text_chunks]
pinecone_index = PC.from_texts(
    docs_chunks,
    embeddings,
    index_name='mr-analyzer'
)

In [32]:
from langchain.chains import RetrievalQA

qa = RetrievalQA.from_chain_type(
    llm=llm_model,
    chain_type="stuff",
    retriever=pinecone_index.as_retriever()
)

In [33]:
response = qa.run(query)
print(response)

The average feedback score for Rag Type A is 3.210526315789474. 

